<a href="https://colab.research.google.com/github/safry4/MASK-RCNN/blob/main/OnData_refined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import skimage.io
import skimage.draw
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import skimage.segmentation as seg
import skimage.color as color
import cv2
from functions import *
from skimage.segmentation import slic
from google.colab.patches import cv2_imshow
from matplotlib.pyplot import figure
from numpy import unravel_index

In [ ]:
def unstack(a, axis=0):
    return np.moveaxis(a, axis, 0)
#    return list(np.moveaxis(a, axis, 0))

In [ ]:
image = skimage.io.imread("DA Lifetime.bmp", as_gray=True)
image = image[:,50:]
skimage.io.imshow(image)

In [ ]:
imageI = skimage.io.imread("DA Intensity.bmp", as_gray=True)
imageI = imageI[:,50:]
skimage.io.imshow(imageI)

In [ ]:
imageD = skimage.io.imread("Donor Lifetime.bmp", as_gray=True)
imageD = imageD[:,50:]
skimage.io.imshow(imageD)

In [ ]:
#noise reduction on Lifetime
blurred = ndimage.gaussian_filter(image, sigma=0.1)
uniform_denoised=ndimage.uniform_filter(blurred,size=3)
plt.imshow(uniform_denoised)
imgu8 = convert(uniform_denoised, 0, 255, np.uint8)

In [ ]:
#noise reduction on Intensity
blurredI = ndimage.gaussian_filter(imageI, sigma=0.1)
uniform_denoisedI=ndimage.uniform_filter(blurredI,size=3)
imgIu8 = convert(uniform_denoisedI, 0, 255, np.uint8)
plt.imshow(imgIu8)

In [ ]:
#no noise reduction
image_with_noise = convert(image, 0, 255, np.uint8)
imageI_with_noise = convert(imageI, 0, 255, np.uint8)

In [ ]:
#Thresholding adjust
from skimage.filters import threshold_otsu
thresh = threshold_otsu(imgIu8)
binary = imgIu8 > 90

fig, axes = plt.subplots(ncols=3, figsize=(24, 7.5))
ax = axes.ravel()
ax[0] = plt.subplot(1, 3, 1)
ax[1] = plt.subplot(1, 3, 2)
ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

ax[0].imshow(imgIu8, cmap=plt.cm.gray)
ax[0].set_title('Original')
ax[0].axis('off')

ax[1].hist(imgIu8.ravel(), bins=256)
ax[1].set_title('Histogram')
ax[1].axvline(thresh, color='r')

ax[2].imshow(binary, cmap=plt.cm.gray)
ax[2].set_title('Thresholded')
ax[2].axis('off')

plt.show()

In [ ]:
# #25% highest intensity
# I_95_percentile= np.percentile(imgIu8, 95)
# I_98_percentile= np.percentile(imgIu8, 98)

# #Threshholding Intensity Image
# to_overlap=imgIu8.copy()
# to_overlap[to_overlap >I_98_percentile] = 255
# #to_overlap[to_overlap <=I25less] = 0
# to_overlap[(to_overlap >=I_95_percentile) & (to_overlap <I_98_percentile)] = 150
# to_overlap[to_overlap <I_95_percentile] = 0
# plt.imshow(to_overlap)

In [ ]:
to_overlap=filter_intensity(image = imageI_with_noise, threshold1 = 70, threshold2 = 50, minArea = 10)
cv2_imshow(to_overlap)

In [ ]:
to_overlap=filter_intensity(image = imgIu8, threshold1 = 120, threshold2 = 65, minArea = 70)
cv2_imshow(to_overlap)

In [ ]:
#multiple threshholds

ProteinFunction=ConnectedComponentFiltered(image=imgu8, threshold = 150, minArea=50)
contours, hierarchy = cv2.findContours(image=ProteinFunction, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)

image_copy = to_overlap.copy()
image_copy = cv2.cvtColor(image_copy, cv2.COLOR_GRAY2BGR) 
cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 100, 255), thickness=1, lineType=cv2.LINE_AA)

###
# ProteinFunction=ConnectedComponentFiltered(image=imgu8, threshold = 150, minArea=5)
# contours, hierarchy = cv2.findContours(image=ProteinFunction, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

# image_copy1 = to_overlap.copy()
# cv2.drawContours(image=image_copy1, contours=contours, contourIdx=-1, color=(255, 0, 255), thickness=1, lineType=cv2.LINE_AA)

###
ProteinFunction=ConnectedComponentFiltered(image=imgu8, threshold = 150, minArea=150)
contours, hierarchy = cv2.findContours(image=ProteinFunction, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
mask = np.zeros(ProteinFunction.shape, np.uint8)
mask_inv = cv2.bitwise_not(mask)
image_copy2=mask
cv2.drawContours(image=image_copy2, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=1, lineType=cv2.LINE_AA)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(50,20))
ax1.imshow(image_copy)
ax2.imshow(image_copy2)

# f, ax = plt.subplots(3,figsize=(100,30))
# ax[0].imshow(image_copy)
# ax[1].imshow(image_copy1)
# ax[2].imshow(image_copy2)

In [ ]:
#iterate over countours

###
ProteinFunction=ConnectedComponentFiltered(image=imgu8, threshold = 150, minArea=50)
contours, hierarchy = cv2.findContours(image=ProteinFunction, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
#stores the (x,y) coordinates of the boundary of a shape
cnt = sorted(contours, key=cv2.contourArea, reverse=True)
l=[cnt[1]]

mask = np.zeros(ProteinFunction.shape, np.uint8)
mask_inv = cv2.bitwise_not(mask)
image_copy2=mask
draw=cv2.drawContours(image=image_copy2, contours=l, contourIdx=0, color=(255, 0, 0), thickness=-1, lineType=cv2.LINE_AA)
#cv2_imshow(image_copy2)

In [ ]:
#Countours

#Lifetime for each countour
ProteinFunction=ConnectedComponentFiltered(image=imgu8, threshold = 150, minArea=250)
contours, hierarchy = cv2.findContours(image=ProteinFunction, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
cnt = sorted(contours, key=cv2.contourArea, reverse=True)
l1, l2 =[cnt[12]], [cnt[1]]


image_copy1 = imgu8.copy()
draw=cv2.drawContours(image=image_copy1, contours=l1, contourIdx=-1, color=(0, 0, 0), thickness=1, lineType=cv2.LINE_AA)

image_copy2 = imgu8.copy()
draw=cv2.drawContours(image=image_copy2, contours=l2, contourIdx=-1, color=(0, 0, 0), thickness=1, lineType=cv2.LINE_AA)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30,15))
ax1.imshow(image_copy1)
ax2.imshow(image_copy2)


ProteinFunction=ConnectedComponentFiltered(image=imgu8, threshold = 150, minArea=250)
contours, hierarchy = cv2.findContours(image=ProteinFunction, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
cnt = sorted(contours, key=cv2.contourArea, reverse=True)
l1, l2 =[cnt[12]], [cnt[1]]
image_copy1 = imgu8.copy()
cv2.drawContours(image=image_copy1, contours=l1, contourIdx=-1, color=(255, 255, 255), thickness=-1, lineType=cv2.LINE_AA)

image_copy2 = imgu8.copy()
cv2.drawContours(image=image_copy2, contours=l2, contourIdx=-1, color=(255, 255, 255), thickness=-1, lineType=cv2.LINE_AA)

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(40,15))
# ax1.imshow(image_copy1)
# ax2.imshow(image_copy2)


x1=np.where(image_copy1 ==255)[0]
y1=np.where(image_copy1 ==255)[1]
Lifetime1=imgu8[x1,y1]

x2=np.where(image_copy2 ==255)[0]
y2=np.where(image_copy2 ==255)[1]
Lifetime2=imgu8[x2,y2]

##
#Intensity
ProteinFunction=ConnectedComponentFiltered(image=imgu8, threshold = 150, minArea=250)
contours, hierarchy = cv2.findContours(image=ProteinFunction, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
cnt = sorted(contours, key=cv2.contourArea, reverse=True)
l1, l2 =[cnt[12]], [cnt[1]]
#9,12
image_copy1 = imgIu8.copy()
cv2.drawContours(image=image_copy1, contours=l1, contourIdx=-1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
#cv2_imshow(image_copy)

image_copy2 = imgIu8.copy()
cv2.drawContours(image=image_copy2, contours=l2, contourIdx=-1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
#cv2_imshow(image_copy)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30,15))
ax1.imshow(image_copy1)
ax2.imshow(image_copy2)
##

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Lifetime of each countour', fontsize=16)
ax1.hist(Lifetime1)
ax2.hist(Lifetime2)
plt.plot()

##

##

#Intensity for each countour
ProteinFunction=ConnectedComponentFiltered(image=imgu8, threshold = 150, minArea=250)
contours, hierarchy = cv2.findContours(image=ProteinFunction, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
cnt = sorted(contours, key=cv2.contourArea, reverse=True)
l1, l2 =[cnt[12]], [cnt[1]]

#imgIu8 = cv2.cvtColor(imgIu8,cv2.COLOR_RGB2GREY)

image_copy1 = imgIu8.copy()
cv2.drawContours(image=image_copy1, contours=l1, contourIdx=-1, color=(255, 255, 255), thickness=-1, lineType=cv2.LINE_AA)

x1=np.where(image_copy1 ==255)[0]
y1=np.where(image_copy1 ==255)[1]
Intensity1=imgIu8[x1,y1]

image_copy2 = imgIu8.copy()
cv2.drawContours(image=image_copy2, contours=l2, contourIdx=-1, color=(255, 255, 255), thickness=-1, lineType=cv2.LINE_AA)

x2=np.where(image_copy2 ==255)[0]
y2=np.where(image_copy2 ==255)[1]
Intensity2=imgIu8[x2,y2]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Intensity of each countour', fontsize=16)
ax1.hist(Intensity1)
ax2.hist(Intensity2)
plt.plot()

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))


In [ ]:
#testing
image = imgIu8.copy()
image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
image = cv2.circle(image, (185,246), radius=1, color=(0, 100, 255), thickness=5)
draw=cv2.drawContours(image=image, contours=l1, contourIdx=-1, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
cv2_imshow(image)

In [ ]:
#testing

In [ ]:
#totry then threshold() (or Sobel() or Canny() or inRange()

In [ ]:
#FRET

In [ ]:
image = skimage.io.imread("DA Lifetime.bmp", as_gray=True)
image = image[:,50:]
skimage.io.imshow(image)

imageD = skimage.io.imread("Donor Lifetime.bmp", as_gray=True)
imageD = imageD[:,50:]
skimage.io.imshow(imageD)

In [ ]:
DA_Intensity = skimage.io.imread("DA Intensity.bmp", as_gray=True)
DA_Intensity = DA_Intensity[:,50:]
skimage.io.imshow(DA_Intensity)

In [ ]:
A_Intensity = skimage.io.imread("Acceptor Intensity.bmp", as_gray=True)
A_Intensity = A_Intensity[:,50:]
skimage.io.imshow(A_Intensity)

In [ ]:
DA_Lifetime=image.copy()
D_Lifetime=imageD.copy()

In [ ]:
#to_grey 0..255
DA_Lifetime = convert(DA_Lifetime, 0, 255, np.uint8)
D_Lifetime = convert(D_Lifetime, 0, 255, np.uint8)

In [ ]:
skimage.io.imshow(DA_Lifetime)

In [ ]:
skimage.io.imshow(D_Lifetime)

In [ ]:
with np.errstate(divide='ignore', invalid='ignore'):
    FRET = np.true_divide(DA_Lifetime,D_Lifetime)
    FRET[FRET == np.inf] = 0
    FRET = np.nan_to_num(FRET)

In [ ]:
skimage.io.imshow(FRET)

In [ ]:
#Coincidence

In [ ]:
DA_Intensity = skimage.io.imread("DA Intensity.bmp", as_gray=True)
DA_Intensity = DA_Intensity[:,50:]
skimage.io.imshow(DA_Intensity)

In [ ]:
A_Intensity = skimage.io.imread("Acceptor Intensity.bmp", as_gray=True)
A_Intensity = A_Intensity[:,50:]
skimage.io.imshow(A_Intensity)

In [ ]:
#to_grey 0..255
DA_Intensity = convert(DA_Intensity, 0, 255, np.uint8)
A_Intensity = convert(A_Intensity, 0, 255, np.uint8)

In [ ]:
to_overlap=filter_intensity(image = DA_Intensity, threshold1 = 45, threshold2 = 45, minArea = 70)
cv2_imshow(to_overlap)

In [ ]:
#set thresholds for D_Intensity, extract countours

In [ ]:
#just to see
contours, hierarchy = cv2.findContours(image=to_overlap, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
# cnt = sorted(contours, key=cv2.contourArea, reverse=True)
# l1, l2 =[cnt[12]], [cnt[1]]


image_copy1 = DA_Intensity.copy()
cv2.drawContours(image=image_copy1, contours=contours, contourIdx=-1, color=(0, 100, 0), thickness=1, lineType=cv2.LINE_AA)

figure(figsize=(50, 40), dpi=80)
plt.imshow(image_copy1)

In [ ]:
#the same as previous to extract coordinates from D_Intensity
contours, hierarchy = cv2.findContours(image=to_overlap, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
# cnt = sorted(contours, key=cv2.contourArea, reverse=True)
# l1, l2 =[cnt[12]], [cnt[1]]


image_copy1 = DA_Intensity.copy()
cv2.drawContours(image=image_copy1, contours=contours, contourIdx=-1, color=(255, 255, 255), thickness=-1, lineType=cv2.LINE_AA)

figure(figsize=(10, 8), dpi=80)
plt.imshow(image_copy1)

In [ ]:
#extract coordinates of white regions
x1=np.where(image_copy1 ==255)[0]
y1=np.where(image_copy1 ==255)[1]
#extract values
#Intensity1=DA_Intensity[x1,y1]

In [ ]:
#Extract from A Intensity

A_Intensity_filtered=filter_intensity(image = A_Intensity, threshold1 = 80, threshold2 = 80, minArea = 70)
cv2_imshow(A_Intensity_filtered)

In [ ]:
#just to see
contours, hierarchy = cv2.findContours(image=A_Intensity_filtered, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
# cnt = sorted(contours, key=cv2.contourArea, reverse=True)
# l1, l2 =[cnt[12]], [cnt[1]]


image_copy3 = A_Intensity.copy()
draw=cv2.drawContours(image=image_copy3, contours=contours, contourIdx=-1, color=(0, 100, 0), thickness=1, lineType=cv2.LINE_AA)

figure(figsize=(50, 40), dpi=80)
plt.imshow(image_copy3)

In [ ]:
#to extract coordinates
#just to see
contours, hierarchy = cv2.findContours(image=A_Intensity_filtered, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
# cnt = sorted(contours, key=cv2.contourArea, reverse=True)
# l1, l2 =[cnt[12]], [cnt[1]]


image_copy2 = A_Intensity.copy()
draw=cv2.drawContours(image=image_copy2, contours=contours, contourIdx=-1, color=(255, 255, 255), thickness=-1, lineType=cv2.LINE_AA)

figure(figsize=(10, 8), dpi=80)
plt.imshow(image_copy2)

In [ ]:
#extract coordinates of white regions
x2=np.where(image_copy2 ==255)[0]
y2=np.where(image_copy2 ==255)[1]
#extract values
#Intensity1=DA_Intensity[x1,y1]

In [ ]:
#find where x1y1 coincide with x2y2, map to FRET

In [ ]:
x1,y1=np.where(image_copy1 ==255)
x2,y2=np.where(image_copy2 ==255)

arr1 = np.stack((x1, y1), axis=1)
arr2 = np.stack((x2, y2), axis=1)

aset = set([tuple(x) for x in arr1])
bset = set([tuple(x) for x in arr2])
coordinates = np.array([x for x in aset & bset])

x,y = unstack(coordinates, axis=1)

img = np.zeros_like(image_copy2)

img[x, y] = 1

In [ ]:
skimage.io.imshow(img)

In [ ]:
#just to see
#img = np.zeros_like(image_copy2)
img = convert(img, 0, 255, np.uint8)
contours, hierarchy = cv2.findContours(image=img, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
cnt = sorted(contours, key=cv2.contourArea, reverse=True)
# l1, l2 =[cnt[12]], [cnt[1]]


image_copy2 = FRET.copy()
draw=cv2.drawContours(image=image_copy2, contours=cnt, contourIdx=-1, color=(20, 20, 20), thickness=1, lineType=cv2.LINE_AA)

figure(figsize=(10, 8), dpi=80)
plt.imshow(image_copy2)

In [ ]:
FRET[410,200]

In [ ]:
#How FRET is distributed inside

In [ ]:
img = convert(img, 0, 255, np.uint8)
contours, hierarchy = cv2.findContours(image=img, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
cnt = sorted(contours, key=cv2.contourArea, reverse=True)
# l1, l2 =[cnt[12]], [cnt[1]]


#image_copy2 = FRET.copy()
image_copy2 = np.zeros_like(image_copy2)
draw=cv2.drawContours(image=image_copy2, contours=cnt, contourIdx=-1, color=(255, 255, 255), thickness=-1, lineType=cv2.LINE_AA)

figure(figsize=(10, 8), dpi=80)
plt.imshow(image_copy2)

In [ ]:
figure(figsize=(10, 8), dpi=80)
plt.imshow(FRET)

In [ ]:
xf=np.where(image_copy2 ==255)[0]
yf=np.where(image_copy2 ==255)[1]

FRET_hist=FRET[xf,yf]
FRET_non_zero = [i for i in FRET_hist if i > 0]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Intensity of each countour', fontsize=16)
ax1.hist(FRET_hist)
ax2.hist(FRET_non_zero)
plt.plot()

In [ ]:
FRET_intersec = np.zeros_like(FRET)
FRET_intersec[xf,yf]=FRET[xf,yf]
F = FRET_intersec.copy()
#image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
F = cv2.circle(F, (337,301), radius=1, color=(25, 25, 25), thickness=5)
figure(figsize=(10, 8), dpi=80)
plt.imshow(F)